In [1]:
%pip install numpy pandas pillow scikit-learn kagglehub matplotlib ipython
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!conda install -c conda-forge background-generator -y


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Looking in indexes: https://download.pytorch.org/whl/cu121


'conda' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import os
import pandas as pd
import numpy as np
import math
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader, Subset, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA version:", torch.version.cuda)
    print("Device name:", device)
else:
    device = torch.device("cpu")
    print("CUDA is not available")
    print("Device name:", device)


CUDA is not available
Device name: cpu


In [4]:
class Display:
    @staticmethod
    def image_Display(image):
        if torch.is_tensor(image):
            image = image.detach().cpu().numpy()

        if image.ndim == 1:
            image = image.reshape(28, 28)

        image = image.astype('uint8')  # ensure proper display format

        display(Image.fromarray(image, mode='L'))


In [ ]:
# def convert_merged_csv_to_pt(train_csv_path, test_csv_path, pt_path):
#     dtype_map = {'label': 'int64'} | {str(i): 'uint8' for i in range(1, 785)}

#     df_train = pd.read_csv(train_csv_path, dtype=dtype_map, low_memory=False)
#     df_test = pd.read_csv(test_csv_path, dtype=dtype_map, low_memory=False)

#     df = pd.concat([df_train, df_test], ignore_index=True).sample(frac=1, random_state=42)

#     pixel_array = df.drop(columns='label').to_numpy(dtype='float32', copy=True)
#     label_array = df['label'].to_numpy(dtype='int64', copy=True)

#     pixels = torch.from_numpy(pixel_array)
#     labels = torch.from_numpy(label_array)

#     torch.save((pixels, labels), pt_path)
#     print(f"✅ Saved: {pt_path}")

# convert_merged_csv_to_pt("Dataset/mnist_train.csv", "Dataset/mnist_test.csv", "Dataset/mnist.pt")

In [6]:
class Dataset(Dataset):
    def __init__(self, pt_path):
        self.pixels, self.labels = torch.load(pt_path, weights_only=True)

    def __getitem__(self, index):
        return self.pixels[index], self.labels[index]

    def __len__(self):
        return self.labels.size(0)

In [ ]:
# def benchmark_workers(pt_path="Dataset/mnist.pt", max_workers=12, batch_size=512):
#     dataset = Dataset(pt_path)
#     indices = np.arange(len(dataset))
#     train_idx, _ = train_test_split(indices, test_size=0.2, shuffle=True)
#     train_set = Subset(dataset, train_idx)

#     print(f"\nBenchmarking num_workers from 0 to {max_workers}...\n")

#     for nw in range(max_workers + 1):
#         loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,
#                                              num_workers=nw, pin_memory=True,
#                                              persistent_workers=(nw > 0))
#         torch.cuda.synchronize() if torch.cuda.is_available() else None
#         t0 = time.perf_counter()

#         for x, y in loader:
#             pass  # simulate one epoch

#         torch.cuda.synchronize() if torch.cuda.is_available() else None
#         t1 = time.perf_counter()
#         print(f"num_workers={nw:<2} | time: {t1 - t0:.3f} sec")

# benchmark_workers() 

In [8]:
class Dataloaders:
    def __init__(self, dataset_cls):
        self.dataset_cls = dataset_cls

    def get(self, pt_path, batch_size=None, test_size=0.09, num_workers=0):
        dataset = self.dataset_cls(pt_path)

        indices = np.arange(len(dataset))
        train_idx, test_idx = train_test_split(indices, test_size=test_size, shuffle=True)

        train_set = Subset(dataset, train_idx)
        test_set = Subset(dataset, test_idx)

        effective_batch_size = len(train_set) if batch_size is None else batch_size

        train_loader = DataLoader(train_set, batch_size=effective_batch_size, shuffle=True,
                                  num_workers=num_workers, pin_memory=True,
                                  persistent_workers=(num_workers > 0))

        test_loader = DataLoader(test_set, batch_size=len(test_set) if batch_size is None else batch_size,
                                 shuffle=False, num_workers=num_workers, pin_memory=True,
                                 persistent_workers=(num_workers > 0))

        return train_loader, test_loader


In [9]:
loaders = Dataloaders(Dataset)

train_loader, test_loader = loaders.get("Dataset/mnist.pt", batch_size=None)

for x, y in train_loader:
    x = x.to(device, non_blocking=True)
    y = y.to(device, non_blocking=True)

    Display.image_Display(x[1])
    print("Label:", y[1].item())

    print(x.shape, y.shape)
    break


C:\Users\ma\AppData\Local\Temp\ipykernel_7096\29157527.py:12: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  display(Image.fromarray(image, mode='L'))


Label: 0
torch.Size([63700, 784]) torch.Size([63700])


In [10]:
class Scaling(nn.Module):
    def __init__(self, input_tensor: torch.Tensor, bounds=(0.0, 255.0)):
        super().__init__()

        self.device = input_tensor.device
        self.input_dtype = input_tensor.dtype

        min_val = input_tensor.min()
        max_val = input_tensor.max()

        lower, upper = map(float, bounds)
        delta_mm = max_val - min_val
        delta_ul = upper - lower

        a = delta_mm + delta_ul
        b = lower * delta_mm + min_val * delta_ul
        c = 2 * delta_mm * delta_ul

        self.register_buffer("a", a)
        self.register_buffer("b", b)
        self.register_buffer("c", c)

    def forward(self, input_tensor: torch.Tensor):
        if self.c == 0: return input_tensor
        return (self.a * input_tensor - self.b) / self.c

    @torch.no_grad()
    def inverse(self, scaled_tensor: torch.Tensor):
        if self.c == 0: return scaled_tensor
        return (self.c * scaled_tensor + self.b) / self.a


In [11]:
class Classifier(nn.Module):
    def __init__(self, categories: int):
        super().__init__()
        self.categories = categories

    def forward(self, labels: torch.Tensor) -> torch.Tensor:
        return F.one_hot(labels.view(-1), num_classes=self.categories).float()

    @torch.no_grad()
    def inverse(self, one_hot_tensor: torch.Tensor) -> torch.Tensor:
        return one_hot_tensor.argmax(dim=1, keepdim=True)


In [12]:
for x, y in train_loader:
    x = x.to(device, non_blocking=True)
    y = y.to(device, non_blocking=True)

    Display.image_Display(x[1])
    print("Label:", y[1].item())

    bounds = (0.0, 255.0)
    Scaling1 = Scaling(x, bounds)
    Classifier1 = Classifier(10).to(device, non_blocking=True)

    x = Scaling1(x)
    y = Classifier1(y)
    x_inverse = Scaling1.inverse(x)
    y_declassified = Classifier1.inverse(y)
    Display.image_Display(x_inverse[1])
    print("Declassified label:", y_declassified[1].item())

    print(x.shape, y.shape)
    break

C:\Users\ma\AppData\Local\Temp\ipykernel_7096\29157527.py:12: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  display(Image.fromarray(image, mode='L'))


Label: 6


Declassified label: 6
torch.Size([63700, 784]) torch.Size([63700, 10])


In [ ]:
# class NN(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.layer_sizes = [784, 107, 26, 10]
#         self.net = nn.Sequential(
#             nn.Linear(self.layer_sizes[0], self.layer_sizes[1], bias=True),
#             nn.Sigmoid(),                       # fused activation
#             nn.Linear(self.layer_sizes[1], self.layer_sizes[2], bias=True),
#             nn.ReLU(inplace=True),              # in‑place
#             nn.Linear(self.layer_sizes[2], self.layer_sizes[3], bias=True),
#             nn.Softmax(dim=1)
#         )

#         nn.init.xavier_uniform_(self.net[0].weight)
#         nn.init.uniform_(self.net[0].bias, -1.0/math.sqrt(self.layer_sizes[0]), 1.0/math.sqrt(self.layer_sizes[0]))
#         nn.init.kaiming_normal_(self.net[2].weight, nonlinearity='relu')
#         nn.init.uniform_(self.net[2].bias, -1.0/math.sqrt(self.layer_sizes[1]), 1.0/math.sqrt(self.layer_sizes[1]))
#         nn.init.xavier_normal_(self.net[4].weight)
#         nn.init.uniform_(self.net[4].bias, -1.0/math.sqrt(self.layer_sizes[2]), 1.0/math.sqrt(self.layer_sizes[2]))

#         self.loss_fn = nn.CrossEntropyLoss()

#     def forward(self, x):
#         x = x.contiguous()
#         return self.net(x)

class VisualNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_sizes = [784, 121, 25, 10]

        self.fc1 = nn.Linear(self.layer_sizes[0], self.layer_sizes[1], bias=True)
        self.fc2 = nn.Linear(self.layer_sizes[1], self.layer_sizes[2], bias=True)
        self.fc3 = nn.Linear(self.layer_sizes[2], self.layer_sizes[3], bias=True)

        self.loss_fn = nn.CrossEntropyLoss()
        self.activations = {}

        # Initialize weights & biases
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.uniform_(self.fc1.bias, -1.0 / math.sqrt(self.layer_sizes[0]), 1.0 / math.sqrt(self.layer_sizes[0]))

        nn.init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        nn.init.uniform_(self.fc2.bias, -1.0 / math.sqrt(self.layer_sizes[1]), 1.0 / math.sqrt(self.layer_sizes[1]))

        nn.init.xavier_normal_(self.fc3.weight)
        nn.init.uniform_(self.fc3.bias, -1.0 / math.sqrt(self.layer_sizes[2]), 1.0 / math.sqrt(self.layer_sizes[2]))

    def forward(self, x):
        self.activations['input'] = x.detach().cpu()  # (batch_size, 784)

        x = torch.sigmoid(self.fc1(x))
        self.activations['layer1'] = x.detach().cpu()  # (batch_size, 121)

        x = torch.relu(self.fc2(x))
        self.activations['layer2'] = x.detach().cpu()  # (batch_size, 25)

        x = self.fc3(x)
        self.activations['output'] = x.detach().cpu()  # (batch_size, 10)

        return torch.softmax(x, dim=1)


In [ ]:
import matplotlib.pyplot as plt

def visualize_activations(activations, epoch, sample_idx=0):
    shapes = {
        'input': (28, 28),
        'layer1': (11, 11),
        'layer2': (5, 5),
        'output': (1, 10)
    }

    fig, axs = plt.subplots(1, 4, figsize=(14, 4))

    for i, (layer_name, shape) in enumerate(shapes.items()):
        act = activations[layer_name][sample_idx].view(*shape)
        axs[i].imshow(act, cmap='gray')
        axs[i].set_title(f"{layer_name} ({shape[0]}×{shape[1]})")
        axs[i].axis('off')

    plt.suptitle(f"Epoch {epoch} - Neuron Activations")
    plt.tight_layout()
    plt.show()


In [14]:
for x, y in train_loader:
    x = x.to(device, non_blocking=True)
    y = y.to(device, non_blocking=True)

    Display.image_Display(x[1])
    print("Label:", y[1].item())

    bounds = (0.0, 255.0)
    Scaling1 = Scaling(x, bounds)
    Classifier1 = Classifier(10).to(device, non_blocking=True)
    net = NN().to(device, non_blocking=True)

    x_scaled = Scaling1(x)
    y_encoded = Classifier1(y)

    yHAT = net(x_scaled)
    loss = net.loss_fn(yHAT, y)
    print("CrossEntropy Loss:", loss.item())

    x_inverse = Scaling1.inverse(x_scaled)
    y_declassified = Classifier1.inverse(y_encoded)
    yHAT_declassified = Classifier1.inverse(yHAT)

    Display.image_Display(x_inverse[1])
    print("Actual:", y_declassified[1].item())
    print("Predicted:", yHAT_declassified[1].item())

    print(x_scaled.shape, y_encoded.shape, yHAT.shape)
    break


C:\Users\ma\AppData\Local\Temp\ipykernel_7096\29157527.py:12: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  display(Image.fromarray(image, mode='L'))


Label: 3
CrossEntropy Loss: 2.3056182861328125


Actual: 3
Predicted: 9
torch.Size([63700, 784]) torch.Size([63700, 10]) torch.Size([63700, 10])


In [ ]:
class UltraFastTrainer:
    def __init__(
        self,
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer_cls=torch.optim.Adam,
        lr=1e-3,
        max_epochs=50,
        clip_grad_norm=1.0,
        use_amp=True,
        use_tqdm=False,
        save_path=None
    ):
        self.model = model.to(device)
        self.loss_fn = loss_fn
        self.optimizer = optimizer_cls(model.parameters(), lr=lr)
        self.max_epochs = max_epochs
        self.clip_grad_norm = clip_grad_norm
        self.use_amp = use_amp
        self.save_path = save_path
        self.use_tqdm = use_tqdm

        self.scaler_amp = GradScaler(enabled=use_amp)
        self.loss_history = []

    def _prefetch_loader(self, loader):
        for x, y in loader:
            yield x.to(device, non_blocking=True), y.to(device, non_blocking=True)

    def train(self, train_loader):
        self.model.train()

        x_sample, _ = next(iter(train_loader))
        scaler = Scaling(x_sample.to(device))

        for epoch in range(1, self.max_epochs + 1):
            start = time.perf_counter()
            total_loss = 0.0

            iterable = self._prefetch_loader(train_loader)
            if self.use_tqdm:
                iterable = tqdm(iterable, desc=f"Epoch {epoch}", leave=False)

            for x, y in iterable:
                x = scaler(x)

                self.optimizer.zero_grad(set_to_none=True)

                with autocast(device_type=device.type, enabled=self.use_amp):
                    y_pred = self.model(x)
                    loss = self.loss_fn(y_pred, y)

                if epoch % 1 == 0:  # Show on every epoch
                    visualize_activations(self.model.activations, epoch)

                
                self.scaler_amp.scale(loss).backward()

                if self.clip_grad_norm:
                    self.scaler_amp.unscale_(self.optimizer)
                    nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_grad_norm)

                self.scaler_amp.step(self.optimizer)
                self.scaler_amp.update()

                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)
            self.loss_history.append(avg_loss)

            elapsed = time.perf_counter() - start
            print(f"Epoch {epoch:03d} | Loss: {avg_loss:.4f} | Time: {elapsed:.2f}s")

        if self.save_path:
            torch.save(self.model.state_dict(), self.save_path)

    def evaluate(self, test_loader):
        self.model.eval()
        correct, total = 0, 0

        scaler = Scaling(next(iter(test_loader))[0].to(device))

        with torch.no_grad():
            for x, y in self._prefetch_loader(test_loader):
                x = scaler(x)
                pred = self.model(x).argmax(dim=1)
                correct += (pred == y.view(-1)).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Test Accuracy: {acc*100:.2f}%")
        return acc

    def plot_loss(self):
        plt.plot(self.loss_history)
        plt.title("Loss Curve")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.grid(True)
        plt.tight_layout()
        plt.show()


In [ ]:
model = NN()
trainer = UltraFastTrainer(model, nn.CrossEntropyLoss(), max_epochs=100, use_amp=True)

trainer.train(train_loader)
trainer.evaluate(test_loader)
trainer.plot_loss()


C:\Users\ma\AppData\Roaming\Python\Python310\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Epoch 001 | Loss: 2.3002 | Time: 26.16s
Epoch 002 | Loss: 2.2955 | Time: 26.95s
Epoch 003 | Loss: 2.2905 | Time: 44.97s
Epoch 004 | Loss: 2.2851 | Time: 30.04s
Epoch 005 | Loss: 2.2794 | Time: 31.79s
Epoch 006 | Loss: 2.2734 | Time: 38.39s
Epoch 007 | Loss: 2.2669 | Time: 31.63s
Epoch 008 | Loss: 2.2600 | Time: 31.07s
Epoch 009 | Loss: 2.2529 | Time: 29.04s
Epoch 010 | Loss: 2.2455 | Time: 29.83s
Epoch 011 | Loss: 2.2380 | Time: 29.86s
Epoch 012 | Loss: 2.2302 | Time: 30.65s
Epoch 013 | Loss: 2.2226 | Time: 30.09s
Epoch 014 | Loss: 2.2149 | Time: 29.91s
Epoch 015 | Loss: 2.2077 | Time: 31.03s
Epoch 016 | Loss: 2.2005 | Time: 31.86s
Epoch 017 | Loss: 2.1935 | Time: 31.79s
Epoch 018 | Loss: 2.1865 | Time: 31.05s
Epoch 019 | Loss: 2.1796 | Time: 30.42s
Epoch 020 | Loss: 2.1728 | Time: 29.87s
Epoch 021 | Loss: 2.1664 | Time: 30.68s
Epoch 022 | Loss: 2.1601 | Time: 30.50s
Epoch 023 | Loss: 2.1538 | Time: 30.13s
Epoch 024 | Loss: 2.1476 | Time: 29.92s
Epoch 025 | Loss: 2.1415 | Time: 30.51s
